In [6]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

from collections import defaultdict
import pandas as pd
import numpy as np

from WA_state_API_functions import get_committee_member_data

# Load

In [7]:
engine = create_engine('postgresql://localhost:5432/wa_leg_raw')

In [23]:
vote_df = pd.read_sql_query('select * from "vote_api"',con=engine)

In [28]:
bill_df = pd.read_sql_query('select * from "bill_api"',con=engine)

In [34]:
bill_df['bill_unique'] = bill_df['bill_id'] + ' ' + bill_df['biennium']

In [37]:
len(bill_df['bill_unique'].unique())

78894

In [69]:
bill_df.head()

,biennium,bill_id,class,description,htm_create_date,htm_last_modified_date,htm_url,long_friendly_name,name,type,bill_unique
0,1991-92,HB 1001,Bills,None,1991-08-30T00:00:00,2006-07-10T17:13:53.543,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1001,1001,House Bills,HB 1001 1991-92
1,1991-92,SHB 1001,Bills,None,1991-02-01T00:00:00,2006-07-10T17:13:54.903,http://app.leg.wa.gov/documents/billdocs/1991-...,Substitute House Bill 1001,1001-S,House Bills,SHB 1001 1991-92
2,1991-92,HB 1002,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:11.637,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1002,1002,House Bills,HB 1002 1991-92
3,1991-92,HB 1003,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:11.747,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1003,1003,House Bills,HB 1003 1991-92
4,1991-92,SHB 1003,Bills,None,1991-02-21T00:00:00,2006-07-10T17:14:07.357,http://app.leg.wa.gov/documents/billdocs/1991-...,Substitute House Bill 1003,1003-S,House Bills,SHB 1003 1991-92


In [38]:
vote_df['bill_unique'] = vote_df['bill_id'] + ' ' + vote_df['biennium']

In [40]:
len(vote_df['bill_unique'].unique())

19190

In [52]:
len(vote_df['voter_id'].unique())

530

# Start legislators_df

In [66]:
legislators_df = vote_df.loc[:, ['voter_id', 'voter_name', 'voting_agency']]

In [67]:
legislators_df.drop_duplicates(keep='first', inplace=True)

In [68]:
legislators_df

,voter_id,voter_name,voting_agency
0,7,Anderson,House
1,11,Appelwick,House
2,17,Ballard,House
3,23,Basich,House
4,32,Beck,House
5,34,Belcher,House
6,40,Betrozoff,House
7,45,Bowman,House
8,46,Braddock,House
9,47,Bray,House
